In [1]:
print("OK!")


OK!


In [2]:
from langchain_pinecone import PineconeSparseVectorStore


c:\Users\IdeaPad-320\.conda\envs\chatbotenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
print("OK!")


OK!


In [ ]:
from langchain_pinecone import PineconeSparseVectorStore


c:\Users\IdeaPad-320\.conda\envs\chatbotenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from langchain_community.vectorstores import Pinecone as LangchainPinecone
from pinecone import Pinecone

from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
# from langchain.vectorstores import Pinecone
# import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [ ]:
PINECONE_API_KEY = "pcsk_RzPvd_6VUxvea8gG15euGYNTSghZLFGqZfHq5hcumFHTfhXMeqQFUXGBgBoSmPsYLev2V"
PINECONE_API_ENV="us-east-1-aws"


In [ ]:
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [ ]:
extracted_data = load_pdf("data/")


In [ ]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [ ]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 40000


In [ ]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [ ]:
embeddings = download_hugging_face_embeddings()


C:\Users\IdeaPad-320\AppData\Local\Temp\ipykernel_13288\4238859041.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


KeyboardInterrupt: 

In [ ]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [ ]:
pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)

index_name="medical-chatbot"

#Creating Embeddings for Each of The Text Chunks & storing
# pc = pinecone(api_key=PINECONE_API_KEY)
# # index = pinecone.Index("quickstart")
# index_name = "medical-chatbot"  

# if not pc.has_index(index_name):
#     pc.create_index(
#         name=index_name,
#         dimension=1536,
#         metric="cosine",
#         spec=ServerlessSpec(cloud="aws", region="us-east-1"),
#     )

# index = pc.Index(index_name)



NameError: name 'pinecone' is not defined

In [ ]:
print(type(text_chunks), len(text_chunks))
print(type(text_chunks[0]), text_chunks[0].page_content[:100])
print(type(embeddings))
print(index_name, pinecone.list_indexes())


<class 'list'> 40000
<class 'langchain.schema.document.Document'> The GALE
ENCYCLOPEDIA of
MEDICINE
THIRD EDITION
<class 'langchain.embeddings.huggingface.HuggingFaceEmbeddings'>
medical-chatbot []


In [ ]:
for i in range(0, len(text_chunks), 500):  # batches of 500
    batch = text_chunks[i:i+500]
    print(f"Uploading batch {i} to {i+len(batch)}...")

    Pinecone.from_texts(
        [doc.page_content for doc in batch],
        embedding=embeddings,
        index_name=index_name
    )


Uploading batch 0 to 500...


MaxRetryError: HTTPSConnectionPool(host='controller.us-east-1.pinecone.io', port=443): Max retries exceeded with url: /databases (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000023786122830>: Failed to resolve 'controller.us-east-1.pinecone.io' ([Errno 11001] getaddrinfo failed)"))

In [ ]:
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

ValueError: No active indexes found in your Pinecone project, are you sure you're using the right API key and environment?

In [ ]:
docsearch=Pinecone.from_existing_index(index_name, embeddings)

query = "What are Allergies"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

NameError: name 'Pinecone' is not defined

In [ ]:
if "medical-chatbot" not in pinecone.list_indexes():
    pinecone.create_index(
        name="medical-chatbot",
        dimension=384,  # Change to match your embedding model
        metric="cosine"
    )

ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-04', 'x-cloud-trace-context': 'b86ac900a7db5ef232ba61879d7426fa', 'date': 'Fri, 11 Jul 2025 17:25:56 GMT', 'server': 'Google Frontend', 'Content-Length': '198', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: Request failed. You've reached the max pods allowed in project Default (0). To increase this limit, adjust your project settings in the console. Contact a project owner if you don't have permission.


In [ ]:
print(pinecone.list_indexes())


[]


In [ ]:
from langchain.vectorstores import Pinecone

# Upload chunks in batches of 500
for i in range(0, len(text_chunks), 500):
    batch = text_chunks[i:i+500]
    print(f"📤 Uploading batch {i} to {i + len(batch)}...")

    try:
        docsearch = Pinecone.from_texts(
            [doc.page_content for doc in batch],
            embedding=embeddings,
            index_name=index_name
            # You can optionally add: namespace="default"
        )
    except Exception as e:
        print(f"❌ Failed to upload batch {i}–{i+len(batch)}: {e}")


📤 Uploading batch 0 to 500...
❌ Failed to upload batch 0–500: No active indexes found in your Pinecone project, are you sure you're using the right API key and environment?
📤 Uploading batch 500 to 1000...
❌ Failed to upload batch 500–1000: No active indexes found in your Pinecone project, are you sure you're using the right API key and environment?
📤 Uploading batch 1000 to 1500...
❌ Failed to upload batch 1000–1500: No active indexes found in your Pinecone project, are you sure you're using the right API key and environment?
📤 Uploading batch 1500 to 2000...
❌ Failed to upload batch 1500–2000: No active indexes found in your Pinecone project, are you sure you're using the right API key and environment?
📤 Uploading batch 2000 to 2500...
❌ Failed to upload batch 2000–2500: No active indexes found in your Pinecone project, are you sure you're using the right API key and environment?
📤 Uploading batch 2500 to 3000...
❌ Failed to upload batch 2500–3000: No active indexes found in your Pin

In [3]:
from langchain_community.vectorstores import Pinecone as LangchainPinecone
from pinecone import Pinecone

from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
# from langchain.vectorstores import Pinecone
# import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [4]:
PINECONE_API_KEY = "pcsk_RzPvd_6VUxvea8gG15euGYNTSghZLFGqZfHq5hcumFHTfhXMeqQFUXGBgBoSmPsYLev2V"
PINECONE_API_ENV="us-east-1-aws"


In [5]:
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [ ]:
extracted_data = load_pdf("data/")


In [ ]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [ ]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 40000


In [ ]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [ ]:
embeddings = download_hugging_face_embeddings()


C:\Users\IdeaPad-320\AppData\Local\Temp\ipykernel_13288\4238859041.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


KeyboardInterrupt: 

In [ ]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [ ]:
pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)

index_name="medical-chatbot"

#Creating Embeddings for Each of The Text Chunks & storing
# pc = pinecone(api_key=PINECONE_API_KEY)
# # index = pinecone.Index("quickstart")
# index_name = "medical-chatbot"  

# if not pc.has_index(index_name):
#     pc.create_index(
#         name=index_name,
#         dimension=1536,
#         metric="cosine",
#         spec=ServerlessSpec(cloud="aws", region="us-east-1"),
#     )

# index = pc.Index(index_name)



NameError: name 'pinecone' is not defined

In [ ]:
print(type(text_chunks), len(text_chunks))
print(type(text_chunks[0]), text_chunks[0].page_content[:100])
print(type(embeddings))
print(index_name, pinecone.list_indexes())


<class 'list'> 40000
<class 'langchain.schema.document.Document'> The GALE
ENCYCLOPEDIA of
MEDICINE
THIRD EDITION
<class 'langchain.embeddings.huggingface.HuggingFaceEmbeddings'>
medical-chatbot []


In [ ]:
for i in range(0, len(text_chunks), 500):  # batches of 500
    batch = text_chunks[i:i+500]
    print(f"Uploading batch {i} to {i+len(batch)}...")

    Pinecone.from_texts(
        [doc.page_content for doc in batch],
        embedding=embeddings,
        index_name=index_name
    )


Uploading batch 0 to 500...


MaxRetryError: HTTPSConnectionPool(host='controller.us-east-1.pinecone.io', port=443): Max retries exceeded with url: /databases (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000023786122830>: Failed to resolve 'controller.us-east-1.pinecone.io' ([Errno 11001] getaddrinfo failed)"))

In [ ]:
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

ValueError: No active indexes found in your Pinecone project, are you sure you're using the right API key and environment?

In [ ]:
docsearch=Pinecone.from_existing_index(index_name, embeddings)

query = "What are Allergies"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

NameError: name 'Pinecone' is not defined

In [ ]:
if "medical-chatbot" not in pinecone.list_indexes():
    pinecone.create_index(
        name="medical-chatbot",
        dimension=384,  # Change to match your embedding model
        metric="cosine"
    )

ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-04', 'x-cloud-trace-context': 'b86ac900a7db5ef232ba61879d7426fa', 'date': 'Fri, 11 Jul 2025 17:25:56 GMT', 'server': 'Google Frontend', 'Content-Length': '198', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: Request failed. You've reached the max pods allowed in project Default (0). To increase this limit, adjust your project settings in the console. Contact a project owner if you don't have permission.


In [ ]:
print(pinecone.list_indexes())


[]


In [ ]:
from langchain.vectorstores import Pinecone

# Upload chunks in batches of 500
for i in range(0, len(text_chunks), 500):
    batch = text_chunks[i:i+500]
    print(f"📤 Uploading batch {i} to {i + len(batch)}...")

    try:
        docsearch = Pinecone.from_texts(
            [doc.page_content for doc in batch],
            embedding=embeddings,
            index_name=index_name
            # You can optionally add: namespace="default"
        )
    except Exception as e:
        print(f"❌ Failed to upload batch {i}–{i+len(batch)}: {e}")


📤 Uploading batch 0 to 500...
❌ Failed to upload batch 0–500: No active indexes found in your Pinecone project, are you sure you're using the right API key and environment?
📤 Uploading batch 500 to 1000...
❌ Failed to upload batch 500–1000: No active indexes found in your Pinecone project, are you sure you're using the right API key and environment?
📤 Uploading batch 1000 to 1500...
❌ Failed to upload batch 1000–1500: No active indexes found in your Pinecone project, are you sure you're using the right API key and environment?
📤 Uploading batch 1500 to 2000...
❌ Failed to upload batch 1500–2000: No active indexes found in your Pinecone project, are you sure you're using the right API key and environment?
📤 Uploading batch 2000 to 2500...
❌ Failed to upload batch 2000–2500: No active indexes found in your Pinecone project, are you sure you're using the right API key and environment?
📤 Uploading batch 2500 to 3000...
❌ Failed to upload batch 2500–3000: No active indexes found in your Pin